In [2]:
import numpy as np
import pandas as pd
import os

print(os.listdir("C:/Users/LG/Desktop/CPS dataset"))

['mitm-arpspoofing-1-dec.csv', 'mitm-arpspoofing-2-dec.csv', 'mitm-arpspoofing-3-dec.csv', 'mitm-arpspoofing-4-dec.csv', 'mitm-arpspoofing-5-dec.csv', 'mitm-arpspoofing-6-dec.csv']


In [3]:
import csv
import glob

path='C:/Users/LG/Desktop/CPS dataset/'
merge_path='C:/Users/LG/Desktop/CPS dataset/merge.csv'
file_list=glob.glob(path+'*')

In [11]:
#dataset merge - 헤더가 있는 csv 파일들 합치기
with open(merge_path,'w') as f:
    for i, file in enumerate (file_list):
        if i ==0:
            with open(file,'r') as f2:
                while True:
                    line=f2.readline()
                    if not line:
                        break
                    f.write(line)
        file_name=file.split('\\')[-1]
        print(file.split('\\')[-1]+' write complete...')
    else:
        with open(file,'r') as f2:
            n=0
            while True:
                line = f2.readline()
                if n!=0:
                    f.write(line)
                if not line:
                    break
                n+=1
        file_name=file.split('\\'[-1])
        print(file.split('\\')[-1]+'write complete...')
    print('>>> All file merge complete...')

mitm-arpspoofing-1-dec.csv write complete...
mitm-arpspoofing-2-dec.csv write complete...
mitm-arpspoofing-3-dec.csv write complete...
mitm-arpspoofing-4-dec.csv write complete...
mitm-arpspoofing-5-dec.csv write complete...
mitm-arpspoofing-6-dec.csv write complete...
mitm-arpspoofing-6-dec.csvwrite complete...
>>> All file merge complete...


In [26]:
dataset = pd.read_csv(merge_path)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86792 entries, 0 to 86791
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   frame.number     86792 non-null  int64  
 1   ip.src           86313 non-null  object 
 2   ip.dst           86313 non-null  object 
 3   eth.src          86792 non-null  object 
 4   eth.dst          86792 non-null  object 
 5   arp.src.hw_mac   383 non-null    object 
 6   arp.dst.hw_mac   383 non-null    object 
 7   icmp             20431 non-null  object 
 8   ip.proto         86313 non-null  object 
 9   tcp.srcport      85231 non-null  float64
 10  tcp.dstport      85231 non-null  float64
 11  tcp.len          64879 non-null  float64
 12  tcp.seq          85231 non-null  float64
 13  tcp.ack          64876 non-null  float64
 14  tcp.flags.push   64876 non-null  float64
 15  tcp.flags.reset  64876 non-null  float64
 16  tcp.flags.syn    64876 non-null  float64
 17  tcp.flags.fi

In [14]:
#사용하지 않을 속성 제거
dataset.pop('ip.src')
dataset.pop('ip.dst')
dataset.pop('eth.src')
dataset.pop('eth.dst')
dataset.pop('arp.src.hw_mac')
dataset.pop('arp.dst.hw_mac')
dataset.pop('icmp')
dataset.pop('ip.proto')

0         17
1        NaN
2          1
3          1
4         17
        ... 
86787      6
86788      6
86789      6
86790      6
86791      6
Name: ip.proto, Length: 86792, dtype: object

In [15]:
x=dataset.fillna(value=0)
x=x.astype('float32')

In [16]:
#dataframe shuffle
from sklearn.utils import shuffle
x=shuffle(x)

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

#X:데이터셋, y:타겟
X_len=len(x.columns)-1
y_len=len(x.columns)-1
X=x.iloc[:,0:X_len]
y=x.iloc[:,y_len]

Using TensorFlow backend.


In [18]:
k=20 #fold 수
num_val_samples=len(X)//20

# ANN 생성
classifier = Sequential()

# 첫 번째 게층에 입력 형태를 지정해준다
classifier.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# 두 번째 계층
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu'))

# 세 번째 계층
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# 컴파일을 통해 학습 과정을 구성한다
#optimizer: 최적화기, loss: 손실함수, 모델이 최소화하려고 하는 대상, metrics: 분류 문제이므로 'accuracy'로 설정
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
#val_data: 검증 데이터, val_targets: 검증 타겟 -> (i)~(i+1) 전체 데이터셋 중에 1세트만 가져옴
#partial_train_data: 훈련 데이터, partial_train_targets: 훈련 타겟 -> ~ (i), (i+1)~ 검증 데이터를 제외한 나머지 세트를 가져옴

accuracy_s=[]
precision_rate_s=[]
recall_rate_s=[]
f1_score_s=[]

In [20]:
for i in range(k): #k번 반복
    print('처리중인 폴드 #',i+1)
    
    #검증 데이터
    val_data=X[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=y[i*num_val_samples:(i+1)*num_val_samples]
    
    #훈련 데이터 준비, 다른 분할 전체
    partial_train_data=np.concatenate([X[:i*num_val_samples], X[(i+1)*num_val_samples:]],axis=0)
    partial_train_targets=np.concatenate([y[:i*num_val_samples], y[(i+1)*num_val_samples:]],axis=0)
    
    #Feature Scaling
    partial_train_data=sc.fit_transform(partial_train_data)
    val_data=sc.transform(val_data)
    
    # trainning set을 입력 데이터로 하여 fit함수 사용해 학습
    classifier.fit(partial_train_data, partial_train_targets, batch_size = 40, epochs = 20)
    
    #모델을 사용해 y를 예측함
    y_pred=classifier.predict(val_data)
    y_pred=(y_pred>0.5)
    
    #confusion_matrix 생성
    cm=confusion_matrix(val_targets, y_pred)
    
    #정확도 지표들 계산
    accuracy=(cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[1][0]+cm[0][1])
    precision_rate=cm[1][1]/(cm[1][1]+cm[0][1])
    recall_rate=cm[1][1]/(cm[1][1]+cm[1][0])
    f1_score=cm[1][1]/(cm[1][1]+(cm[1][0]+cm[0][1])/2)
    
    accuracy_s.append(accuracy)
    precision_rate_s.append(precision_rate)
    recall_rate_s.append(recall_rate)
    f1_score_s.append(f1_score)
    
    print('Confusion Matrix : \n',cm)

처리중인 폴드 # 1
Epoch 1/20
82453/82453 [==============================] - 3s 40us/step - loss: 0.2471 - accuracy: 0.9179
Epoch 2/20
82453/82453 [==============================] - 3s 39us/step - loss: 0.1472 - accuracy: 0.9492
Epoch 3/20
82453/82453 [==============================] - 3s 38us/step - loss: 0.1340 - accuracy: 0.9532
Epoch 4/20
82453/82453 [==============================] - 3s 35us/step - loss: 0.1248 - accuracy: 0.9568
Epoch 5/20
82453/82453 [==============================] - 2s 30us/step - loss: 0.1166 - accuracy: 0.9603
Epoch 6/20
82453/82453 [==============================] - 3s 33us/step - loss: 0.1052 - accuracy: 0.9667
Epoch 7/20
82453/82453 [==============================] - 3s 35us/step - loss: 0.0939 - accuracy: 0.9705
Epoch 8/20
82453/82453 [==============================] - 3s 35us/step - loss: 0.0823 - accuracy: 0.9726
Epoch 9/20
82453/82453 [==============================] - 3s 31us/step - loss: 0.0749 - accuracy: 0.9752
Epoch 10/20
82453/82453 [==================

82453/82453 [==============================] - 2s 29us/step - loss: 0.0322 - accuracy: 0.9882
Epoch 12/20
82453/82453 [==============================] - 2s 28us/step - loss: 0.0307 - accuracy: 0.9888
Epoch 13/20
82453/82453 [==============================] - 2s 29us/step - loss: 0.0317 - accuracy: 0.9883
Epoch 14/20
82453/82453 [==============================] - 3s 31us/step - loss: 0.0312 - accuracy: 0.9884
Epoch 15/20
82453/82453 [==============================] - 2s 30us/step - loss: 0.0318 - accuracy: 0.9882
Epoch 16/20
82453/82453 [==============================] - 3s 31us/step - loss: 0.0302 - accuracy: 0.9887
Epoch 17/20
82453/82453 [==============================] - 2s 30us/step - loss: 0.0314 - accuracy: 0.9883
Epoch 18/20
82453/82453 [==============================] - 3s 31us/step - loss: 0.0310 - accuracy: 0.9885
Epoch 19/20
82453/82453 [==============================] - 3s 32us/step - loss: 0.0318 - accuracy: 0.9882
Epoch 20/20
82453/82453 [==============================] -

82453/82453 [==============================] - 5s 65us/step - loss: 0.0282 - accuracy: 0.9894
Epoch 2/20
82453/82453 [==============================] - 3s 42us/step - loss: 0.0282 - accuracy: 0.9894
Epoch 3/20
82453/82453 [==============================] - 3s 42us/step - loss: 0.0284 - accuracy: 0.9892
Epoch 4/20
82453/82453 [==============================] - 3s 39us/step - loss: 0.0285 - accuracy: 0.9893
Epoch 5/20
82453/82453 [==============================] - 3s 36us/step - loss: 0.0280 - accuracy: 0.9897
Epoch 6/20
82453/82453 [==============================] - 3s 40us/step - loss: 0.0280 - accuracy: 0.9895
Epoch 7/20
82453/82453 [==============================] - 4s 54us/step - loss: 0.0281 - accuracy: 0.9894
Epoch 8/20
82453/82453 [==============================] - 4s 50us/step - loss: 0.0288 - accuracy: 0.9890
Epoch 9/20
82453/82453 [==============================] - 4s 53us/step - loss: 0.0281 - accuracy: 0.9892
Epoch 10/20
82453/82453 [==============================] - 4s 52us

In [27]:
print('accuracy : ' , np.mean(accuracy_s))
print('precision : ' , np.mean(precision_rate_s))
print('recall_rate : ', np.mean(recall_rate_s))
print('f1_score : ', np.mean(f1_score_s))

accuracy :  0.9873934086194975
precision :  0.9914211787612988
recall_rate :  0.986961007419745
f1_score :  0.9891767475654314


In [28]:
y_pred=y_pred.astype('int64') #0,1로 출력되도록 boolean->int 형 변환
print('20th Iteration에 대한 target 예측값은 다음과 같다.')
np.set_printoptions(threshold=np.inf,linewidth=np.inf)
print(y_pred.T)

20th Iteration에 대한 target 예측값은 다음과 같다.
[[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1